In [1]:
import typing
from memory import llama_activation
GB = 2**30
TOPS = 1e12
C2C = 200/8  * GB  # per s

print('llama2 7B activations, seq_len=512 ==============>')
llama_activation(head_num=32, kv_head=32, batch_size=1, seq_len=512, hidden_size=4096, \
    immediate_size=11008, layer_num=32, vocab_size=32000, is_print=True)

print('llama2 7B activations, seq_len=4096 ==============>')
llama_activation(head_num=32, kv_head=32, batch_size=1, seq_len=4096, hidden_size=4096, \
    immediate_size=11008, layer_num=32, vocab_size=32000, is_print=True)

print('llama2 70B activations, seq_len=4096==============>')
llama_activation(head_num=64, kv_head=8, batch_size=1, seq_len=4096, hidden_size=8192, \
    immediate_size=28672, layer_num=80, vocab_size=32000, is_print=True, use_flash_attention=True)


print('Qwen2 7B activations, seq_len=512 ==============>')
llama_activation(head_num=28, kv_head=4, batch_size=1, seq_len=512, hidden_size=3584, \
    immediate_size=18944, layer_num=28, vocab_size=152064)

print('Qwen2 7B activations, seq_len=1024 ==============>')
llama_activation(head_num=28, kv_head=4, batch_size=1, seq_len=1024, hidden_size=3584, \
    immediate_size=18944, layer_num=28, vocab_size=152064)

print('Qwen2 7B activations, seq_len=2048 ==============>')
llama_activation(head_num=28, kv_head=4, batch_size=1, seq_len=2048, hidden_size=3584, \
    immediate_size=18944, layer_num=28, vocab_size=152064)
print('Qwen2 7B activations, seq_len=4096 ==============>')
llama_activation(head_num=28, kv_head=4, batch_size=1, seq_len=4096, hidden_size=3584, \
    immediate_size=18944, layer_num=28, vocab_size=152064)


llama2 7B activations, seq_len=512 ==============>
head_num=32, kv_head=32, batch_size=1, seq_len=512, hidden_size=4096, layer_num=32
One Transformer Block:
QKV input: 0.00390625 GB
Q_proj: 0.00390625 GB
K_proj: 0.00390625 GB
V_proj: 0.00390625 GB
attn_out: 0.00390625 GB
O_proj: 0.00390625 GB
Softmax output: 0.03125 GB
RMSNorm input: 0.015625 GB
MLP up input: 0.00390625 GB
MLP up output: 0.010498046875 GB
MLP gate output: 0.010498046875 GB
MLP down input: 0.010498046875 GB
Activation size of One Transformer Block: 0.105712890625 GB
Total Activation size: 3.45556640625 GB
llama2 7B activations, seq_len=4096 ==============>
head_num=32, kv_head=32, batch_size=1, seq_len=4096, hidden_size=4096, layer_num=32
One Transformer Block:
QKV input: 0.03125 GB
Q_proj: 0.03125 GB
K_proj: 0.03125 GB
V_proj: 0.03125 GB
attn_out: 0.03125 GB
O_proj: 0.03125 GB
Softmax output: 2.0 GB
RMSNorm input: 0.125 GB
MLP up input: 0.03125 GB
MLP up output: 0.083984375 GB
MLP gate output: 0.083984375 GB
MLP down i

70.65234375

In [2]:
def megatron_version(a, b, s, h, L, t=1):
    bytes = s*b*h*L*(10 + 24/t + 5*a*s/h/t)
    return bytes/GB

megatron_version(a=32, b=1, s=4096, h=4096, L=32, t=1)

97.0

In [3]:
print('Qwen2 7B==============>')
params_num = 7.62 * 2**30
weights = 2 * params_num / GB
activations = llama_activation(head_num=28, kv_head=4, batch_size=1, seq_len=2048, hidden_size=3584, \
    immediate_size=18944, layer_num=28, vocab_size=152064, use_flash_attention=True)

lora_r = 64
hidden_size = 3584
immediate_size = 18944
layer_num = 28
head_num =28
kv_head = 4
lora_A_q_k_v_o = hidden_size * lora_r * 2
lora_B_q_o =  lora_r * hidden_size * 2
lora_B_k_v = lora_r * hidden_size/head_num*kv_head * 2
lora_A_up_gate = hidden_size * lora_r * 2
lora_B_up_gate = lora_r * immediate_size * 2
lora_A_down = immediate_size * lora_r * 2
lora_B_down = lora_r * hidden_size * 2
# lora_weights = layer_num*(4 * lora_A_q_k_v_o + 2 * lora_B_q_o + 2 * lora_B_k_v + 3 * lora_A_up_gate + 3 * lora_B_up_gate)/GB
# 最终算法选定的版本，只lora QKVO，没有MLP
lora_weights = layer_num*(4 * lora_A_q_k_v_o + 2 * lora_B_q_o + 2 * lora_B_k_v) / GB
grad = lora_weights
optimizer = lora_weights/2*12
print(f'Weights: {weights} GB')
print(f'Activations: {activations} GB')
print(f'Grad: {grad*1024} MB')
print(f'Lora Weights: {lora_weights*1024} MB')
print(f'Optimizer: {optimizer} GB')
print(f'Total: {weights + activations + grad + lora_weights + optimizer} GB') 

# batchsize = 1->2; 28GB -> 34GB  实测
# 脚本，却是 19->21GB,只增加2.5GB

# 用了flash att是 58GB
# SPDA 是 60多GB
# 不用 是 OutOfMem
#

Qwen2 7B==============>
head_num=28, kv_head=4, batch_size=1, seq_len=2048, hidden_size=3584, layer_num=28
One Transformer Block:
QKV input: 0.013671875 GB
Q_proj: 0.013671875 GB
K_proj: 0.001953125 GB
V_proj: 0.001953125 GB
attn_out: 0.013671875 GB
O_proj: 0.013671875 GB
Softmax output: 0.0059814453125 GB
RMSNorm input: 0.0546875 GB
MLP up input: 0.013671875 GB
MLP up output: 0.072265625 GB
MLP gate output: 0.072265625 GB
MLP down input: 0.072265625 GB
Activation size of One Transformer Block: 0.3497314453125 GB
Total Activation size: 10.99365234375 GB
Weights: 15.24 GB
Activations: 10.99365234375 GB
Grad: 77.0 MB
Lora Weights: 77.0 MB
Optimizer: 0.451171875 GB
Total: 26.835214843750002 GB


length = 512

512*3584  3584,3584


4096*4096*2  = 32MB
32/2 = 16 MB

4096, 2048，  4096， 128

3584/2 = 1792

3584, 1792   3584, 112


计算量统计
读写量统计

In [4]:
def all_reduce(C2C, bytes, device_num):
    comm_time = bytes/device_num/C2C * (device_num-1) * 2
    return comm_time

b , s, h, immediate_size = 1, 4096, 4096, 11008

layer_num = 32
tp = 2
def TP_comm(b, s, h, layer_num):
    print(f'batchsize = {b} >>>>>>>>>>>>>>>')
    comm_byte = b * s * h * 2 + b * s * h * 2
    print(f"TP, Comm bytes: {comm_byte/GB} GB")
    comm_byte_total = comm_byte * layer_num
    print(f"TP, Comm bytes total: {comm_byte_total/GB} GB")
    comm_time = all_reduce(C2C, comm_byte_total, tp)
    print(f"TP, Comm time: {comm_time} s")

print(all_reduce(C2C, 7.62*2*GB, 32))
print(all_reduce(C2C, 0.162*2*GB, 32))
TP_comm(b, s, h, layer_num)
TP_comm(4, s, h, layer_num)




1.1811
0.02511
batchsize = 1 >>>>>>>>>>>>>>>
TP, Comm bytes: 0.0625 GB
TP, Comm bytes total: 2.0 GB
TP, Comm time: 0.08 s
batchsize = 4 >>>>>>>>>>>>>>>
TP, Comm bytes: 0.25 GB
TP, Comm bytes total: 8.0 GB
TP, Comm time: 0.32 s


In [5]:
params_num = 7.62 * 10**9
weights = 2 * params_num
tp = 2
total_devcie = 32
comm_byte = weights/tp
print(f"DP Comm bytes: {comm_byte/GB} GB")
comm_time = all_reduce(C2C, comm_byte, total_devcie/tp)
print(f"DP Comm time: {comm_time} s")

#DP 时间不会随着batchszie变，但是会随着tp变
# 所以增加tp，减少DP时间，同时增大batchsize，是个不错的选择

DP Comm bytes: 7.096678018569946 GB
DP Comm time: 0.532250851392746 s


In [6]:
batch_size = 1
seq_len = 512
layer_num = 28
hidden_size = 3584
Compute = batch_size * seq_len * 2*params_num

Compute_time = Compute/128/TOPS
print(f"Compute time: {Compute_time} s")
Communication = batch_size * seq_len * hidden_size * 2 * 2 * layer_num
Communication_time = Communication/C2C * 2
print(f"Communication: {Communication/GB} GB ")

print(f"Communication time: {Communication_time} s")
print(Compute_time/Communication_time)
print(7*GB/4/hidden_size/layer_num*25/128/1000)

Compute time: 0.06096 s
Communication: 0.19140625 GB 
Communication time: 0.0153125 s
3.981061224489796
3.657142857142857


In [7]:
print(f"lora, DP Comm bytes: {lora_weights} GB")
comm_time = all_reduce(C2C, lora_weights*GB, total_devcie)
print(f"lora, DP Comm time: {comm_time} s")

lora, DP Comm bytes: 0.0751953125 GB
lora, DP Comm time: 0.00582763671875 s


In [8]:
compute_per_token = 2 * params_num
s = 512
b  = 4
total_compute = compute_per_token * s * b
hw_compute = 128 * TOPS
utilization = 1
print(f"Total compute: {total_compute/TOPS} TOPS")
print(f"Compute time: {total_compute/hw_compute/utilization} s")

TP_comm(b, s, h, layer_num)

Total compute: 31.21152 TOPS
Compute time: 0.24384 s
batchsize = 4 >>>>>>>>>>>>>>>
TP, Comm bytes: 0.03125 GB
TP, Comm bytes total: 0.875 GB
TP, Comm time: 0.035 s


In [9]:
s = 4096
total_compute = compute_per_token * s
print(f"Total compute: {total_compute/TOPS} TOPS")
print(f"Compute time: {total_compute/hw_compute} s")

Total compute: 62.42304 TOPS
Compute time: 0.48768 s


In [10]:
# 计算die
# 24*32

56*2*256/8


3584.0

In [11]:
m, k, n = 512, 4096, 4096

operation_intensity = 2 * m * n * k / (2 * m * n + 2* m * k + 2*k * n)
print(operation_intensity)

409.6


In [12]:
def analyze_computation(batch_size, seq_len, vocab_size, hidden_size, head_num, kv_head, immediate_size, layer_num, mlp_style='llama'):
    '''
    count and analyze computations of LLM model training
    Reference:
    'Efficient large-scale language model training on GPU clusters using megatron-LM' 
    https://dl.acm.org/doi/10.1145/3458817.3476209, page11, APPENDIX: FLOATING-POINT OPERATIONS
    
    Parameters:
    - batch_size (int): The batch size.
    - seq_len (int): The sequence length.
    - vocab_size (int): The vocabulary size.
    - hidden_size (int): The hidden size.
    - head_num (int): The number of attention heads.
    - kv_head (int): The number of key-value heads.
    - immediate_size (int): The immediate size, ffn up dim.
    - layer_num (int): The number of layers.
    - mlp_style (str): The style of mlp, 'llama' or 'normal'.

    Returns:
    - int: The total computations (FLOPs).
    '''
    compuations = {}
    # forward pass
    compuations['q_proj'] = batch_size * 2 * seq_len * hidden_size * hidden_size
    compuations['k_proj'] = batch_size * 2 * seq_len * hidden_size * (hidden_size // head_num * kv_head)
    compuations['v_proj'] = batch_size * 2 * seq_len * hidden_size * (hidden_size // head_num * kv_head)
    compuations['att_q_k'] = batch_size * 2 *  seq_len * seq_len * hidden_size
    compuations['att_v'] = batch_size * 2 * seq_len * seq_len * hidden_size
    compuations['o_proj'] = batch_size * 2 * seq_len * hidden_size * hidden_size
    compuations['mlp_up'] = batch_size * 2 * seq_len * hidden_size * immediate_size
    compuations['mlp_down'] = batch_size * 2 * seq_len * immediate_size * hidden_size
    
    if mlp_style == 'llama':        
        compuations['mlp_gate'] = batch_size * 2 * seq_len * hidden_size * immediate_size
    else:
        compuations['mlp_gate'] = 0
    # multiply layer_num
    for k, v in compuations.items():
        compuations[k] = v * layer_num

    # add lm_head
    compuations['lm_head'] = batch_size * 2 * seq_len * hidden_size * vocab_size
    # backward pass is two times of forward pass
    for k, v in compuations.items():
        compuations[k] = 3 * v
    compuations['total'] = sum(compuations.values())
    return compuations

# llama2 7B
print('llama2-7B ===============>>>>>>>>>>>>>>>')
llama2_compuations = analyze_computation(batch_size=1, seq_len=4096, vocab_size=32000, hidden_size=4096, head_num=32, kv_head=32, immediate_size=11008, layer_num=32)
for k, v in llama2_compuations.items():
    print(f'{k}: {v/1e12: 0.4f} TOPS')
    
# Qwen2 7B
print('Qwe2-7B ===============>>>>>>>>>>>>>>>')
qwen2_compuations = analyze_computation(batch_size=1, seq_len=4096, vocab_size=152064, hidden_size=3584, head_num=28, kv_head=4, immediate_size=18944, layer_num=28)
for k, v in qwen2_compuations.items():
    print(f'{k}: {v/1e12: 0.4f} TOPS')

# Qwen2 7B
print('Qwe2-7B ===============>>>>>>>>>>>>>>>')
qwen2_compuations = analyze_computation(batch_size=2, seq_len=512, vocab_size=152064, hidden_size=3584, head_num=28, kv_head=4, immediate_size=18944, layer_num=28)
for k, v in qwen2_compuations.items():
    print(f'{k}: {v/1e12: 0.4f} TOPS')

llama2-7B ===============>>>>>>>>>>>>>>>
q_proj:  13.1941 TOPS
k_proj:  13.1941 TOPS
v_proj:  13.1941 TOPS
att_q_k:  13.1941 TOPS
att_v:  13.1941 TOPS
o_proj:  13.1941 TOPS
mlp_up:  35.4592 TOPS
mlp_down:  35.4592 TOPS
mlp_gate:  35.4592 TOPS
lm_head:  3.2212 TOPS
total:  188.7638 TOPS
Qwe2-7B ===============>>>>>>>>>>>>>>>
q_proj:  8.8390 TOPS
k_proj:  1.2627 TOPS
v_proj:  1.2627 TOPS
att_q_k:  10.1018 TOPS
att_v:  10.1018 TOPS
o_proj:  8.8390 TOPS
mlp_up:  46.7207 TOPS
mlp_down:  46.7207 TOPS
mlp_gate:  46.7207 TOPS
lm_head:  13.3939 TOPS
total:  193.9629 TOPS
Qwe2-7B ===============>>>>>>>>>>>>>>>
q_proj:  2.2098 TOPS
k_proj:  0.3157 TOPS
v_proj:  0.3157 TOPS
att_q_k:  0.3157 TOPS
att_v:  0.3157 TOPS
o_proj:  2.2098 TOPS
mlp_up:  11.6802 TOPS
mlp_down:  11.6802 TOPS
mlp_gate:  11.6802 TOPS
lm_head:  3.3485 TOPS
total:  44.0712 TOPS


In [13]:
7*4096*2

57344